# Notebook - 4 - SQL Server
### Topics Covered in this notebook
1. Transactions in SQL Server
2. Transactions ACID Test in SQL Server

In [1]:
USE kudvenkatdb_master;

Commands completed successfully.

Total execution time: 00:00:00.008

## Transactions
### What is a Transaction?
A transaction is a group of commands that change the data stored in a database. A transaction, is treated as a single unit. A transaction ensures that, either all of the commands succeed, or none of them. If one of the commands in the transaction fails, all of the commands fail, and any data that was modified in the database is rolled back. In this way, transactions maintain the integrity of data in a database.

**Transaction processing follows these steps:**
1. Begin a transaction.
2. Process database commands.
3. Check for errors.

        if errors occured,
            rollback the transaction,
        else
            commit the transaction    
**Note**: NOT able to see the un-committed changes

***SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED***

In [2]:
CREATE TABLE tblPhysicalAddress(
    AddressId int PRIMARY key,
    EmployeeNumber int,
    HouseNumber NVARCHAR(30),
    StreetAddress NVARCHAR(30),
    City NVARCHAR(10)
);

CREATE TABLE tblMailingAddress(
    AddressId int PRIMARY key,
    EmployeeNumber int,
    HouseNumber NVARCHAR(30),
    StreetAddress NVARCHAR(30),
    City NVARCHAR(10)
);

Commands completed successfully.

Total execution time: 00:00:00.023

In [ ]:
INSERT INTO tblMailingAddress VALUES(1,101,'#10','King Street','LONDOON')
INSERT INTO tblPhysicalAddress VALUES(1,101,'#10','King Street','LONDOON')

In [6]:
SELECT * FROM tblPhysicalAddress;
SELECT * FROM tblMailingAddress;

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.043

AddressId,EmployeeNumber,HouseNumber,StreetAddress,City
1,101,#10,King Street,LONDOON


AddressId,EmployeeNumber,HouseNumber,StreetAddress,City
1,101,#10,King Street,LONDOON


In [7]:
-- create a stored procedure for updating the City Column

CREATE PROCEDURE spUpdateAddress
as
BEGIN
    BEGIN TRY
        BEGIN TRANSACTION
            UPDATE tblPhysicalAddress SET City = 'London' WHERE AddressId = 1 -- first updating statement

            UPDATE tblMailingAddress SET City = 'London' WHERE AddressId = 1 -- 2nd updating statement
        COMMIT TRANSACTION
        PRINT 'Transaction Committed'
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION
        PRINT 'Transaction Rolled Back'
    END CATCH
END


Commands completed successfully.

Total execution time: 00:00:00.006

In [8]:
EXEC spUpdateAddress

(1 row affected)

(1 row affected)

Transaction Committed

Total execution time: 00:00:00.010

In [9]:
SELECT * FROM tblPhysicalAddress;
SELECT * FROM tblMailingAddress;

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.020

AddressId,EmployeeNumber,HouseNumber,StreetAddress,City
1,101,#10,King Street,London


AddressId,EmployeeNumber,HouseNumber,StreetAddress,City
1,101,#10,King Street,London


In [10]:
-- here we intentionally added the error to watch the status as  'Transaction Rolled Back'
ALTER PROCEDURE spUpdateAddress
as
BEGIN
    BEGIN TRY
        BEGIN TRANSACTION
            UPDATE tblPhysicalAddress SET City = 'London' WHERE AddressId = 1 -- first updating statement

            UPDATE tblMailingAddress SET City = 'London LONDON' WHERE AddressId = 1 -- 2nd updating statement
        COMMIT TRANSACTION
        PRINT 'Transaction Committed'
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION
        PRINT 'Transaction Rolled Back'
    END CATCH
END

Commands completed successfully.

Total execution time: 00:00:00.026

In [11]:
EXEC spUpdateAddress

(1 row affected)

(0 rows affected)

Transaction Rolled Back

Total execution time: 00:00:00.009

## Transaction ACID Test

**A transaction is a group of database commands that are treated as a single unit.** A successful transaction must pass the "ACID" test, that is, it must be


**Atomic**-All statements in the transaction either completed successfully or they were all rolled back. The task that the set of operations represents is either accomplished or not, but in any case not left half-done.


**Consistent** - All data touched by the transaction is left in a logically consistent state. For example, if stock available numbers are decremented from tblProductTable, then, there has to be a related entry in tblProductSales table. The inventory can't just disappear.

**Isolated:** The transaction must affect data without interfering with other concurrent transactions, or being interfered with by them. This prevents transactions from making changes to data based on uncommitted information, for example changes to a record that are subsequently rolled back. Most databases use locking to maintain transaction isolation.


**Durable:** Once a change is made, it is permanent. If a system error or power failure occurs before a set of commands is complete, those commands are undone and the data is restored to its original state once the system begins running again.